# Testing different SA methods 6/6
## scikit's multinomial naive bayes

In [4]:
# Imports
import csv
import pickle
import random
import time

import numpy as np

from gensim import corpora
from sklearn.naive_bayes import MultinomialNB

# Ugly hackery, but necessary: stackoverflow.com/questions/4383571/importing-files-from-different-folder
import sys
sys.path.append('../../../')
    
from src.streaming import spark_functions

In [5]:
preprocess = spark_functions.preprocessor()
tokenize = spark_functions.tokenizer()

with open('./../../../data/interim/sanders_hydrated.csv') as csv_file:
    iterator = csv.reader(csv_file, delimiter=',')
    # Load the parts we need and preprocess as well as tokenize the text
    tweets = [(tokenize(preprocess(text)), sentiment) for (topic, sentiment, id, text) in iterator]
    # The tweets are ordered, so shuffle for the training/test split
    random.shuffle(tweets)

In [6]:
tweets[0]

(['android',
  'app',
  'beam',
  'bit',
  'galaxy',
  'google',
  'improvements',
  'nexus',
  'nexus',
  'nfc',
  'samsung',
  'sharing',
  'using'],
 'neutral')

In [7]:
"There are %d tweets in this dataset" % len(tweets)

'There are 4347 tweets in this dataset'

In [22]:
def test(classifier, X,Y):
    labeled_correctly = 0
    for index,vector in enumerate(X):
        if classifier.predict([vector]) == Y[index]:
            labeled_correctly+=1
    print("%d/%d = %.2d percent" % (labeled_correctly, len(X), labeled_correctly/len(X)*100))
    
# Use the dictionary we already created for the topic model
# With its own dict, the accuracy on the sanders data improves significantly
dictionary = corpora.Dictionary.load('./../../../data/processed/tweets_sanders.dict')
print("Dictionary has length %d" % len(dictionary))

# This is different from the one used with the nltk methods
def extract_features(document):
    document_words = set(document)
    features = []
    for word in dictionary.token2id:
        features.append(word in document_words)
    return features

Dictionary has length 3961


In [23]:
# Split between training and test set
# 90% training, 10% test
threshold = int(0.9 * len(tweets))
X_train = np.array([extract_features(text) for (text,sentiment) in tweets[:threshold]])
Y_train = np.array([sentiment for (text,sentiment) in tweets[:threshold]])
X_test = np.array([extract_features(text) for (text,sentiment) in tweets[threshold:]])
Y_test = np.array([sentiment for (text,sentiment) in tweets[threshold:]])

print(X_train.shape)
print(Y_train.shape)

(3912, 3961)
(3912,)


Train and test the model with the dict from the streamed tweets and from the sanders dataset to show the descrepancies

In [24]:
t0 = time.time()
classifier = MultinomialNB()
classifier.fit(X_train, Y_train)
print("Training took %d seconds" % (time.time() - t0))
test(classifier, X_test,Y_test)

Training took 0 seconds
326/435 = 74 percent


In [25]:
# Try the same thing with the other dictionary
dictionary = corpora.Dictionary.load('./../../../data/processed/tweets_stream.dict')
print("Dictionary has length %d" % len(dictionary))

# The feature vectory obviously need to be redone
X_train = np.array([extract_features(text) for (text,sentiment) in tweets[:threshold]])
X_test = np.array([extract_features(text) for (text,sentiment) in tweets[threshold:]])

t0 = time.time()
classifier = MultinomialNB()
classifier.fit(X_train, Y_train)
print("Training took %d seconds" % (time.time() - t0))
test(classifier, X_test,Y_test)

Dictionary has length 4929
Training took 0 seconds
272/435 = 62 percent


This is also not very good (and weirdly similar to the nltk results) --> Maybe create another training set with smileys as labels after all 

In [30]:
# Save the classifier if it's any good
save_classifier = open("./../../../models/naive_bayes/scikit_naive_bayes_multinomial.pickle","wb")
pickle.dump(classifier, save_classifier)
save_classifier.close()